## Validación Base de Datos de SNPs

**1.-**  Lectura de la lista de muestras de validación (final_output_file) y crea un set.

**2.-**  Búsqueda directa de la carpeta de cada muestra en samples_directory.

**3.-**  Carga y filtrado del archivo snps.all.ivar.tsv original de cada muestra.

**4.-**  Comparación de los SNPs filtrados con los de la BD (df_filtered_SNPs) usando merge por POS, REF, ALT.

**5.-**  Adición de una columna CHECK_SAMPLES, solo si el sample_id aparece en la columna 'SAMPLES' de la BD.

**6.-**  Adición de una columna de conteo CHECK_SAMPLES_COUNT.

**7.-**  Guardado de un archivo TSV por muestra con los SNPs validados en output_directory.

**8.-**  Comparación del número de SNPs por muestra generados tras la validación con los de su archivo tsv original.

In [ ]:
import pandas as pd
import os
import random
import csv
import sys

# Aumentar el límite de campo de lectura csv al usar engine = 'python' para evitar error
csv.field_size_limit(sys.maxsize)

# Fijar semilla de números aleatorios
random.seed(0)

packs_directory = "/home/laura/andrea/scripts/TB_DDBB_Building/Paquetes_500_muestras"
samples_directory = "/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants"
validation_directory = "/home/laura/andrea/scripts/TB_DDBB_Building/Validacion"
output_directory = "/home/laura/andrea/scripts/TB_DDBB_Building/Validacion/Validation_Samples_Files"

DDBB_file = "/home/laura/andrea/scripts/TB_DDBB_Building/Filtrado_SNPs/Filtered_SNPs_20250401_012139_raw_withComplex.tsv"

# DDBB en dataframe
df_filtered_SNPs = pd.read_csv(DDBB_file, sep='\t')

In [14]:
# Listado total de SNPs
total_SNPs = len(df_filtered_SNPs)
print("Número total de SNPs recogidos en la BBDD:")
print(total_SNPs)
print("--------------------------------------------------")
# Cálculo del 1% de SNPs totales de la BBDD
sampling_SNPs = max(1, int(0.01 * total_SNPs)) 
print("1% del total de SNPS:")
print(sampling_SNPs)

Número total de SNPs recogidos en la BBDD:
2264944
--------------------------------------------------
1% del total de SNPS:
22649


In [15]:
# Hacer listado total de muestras
dir_list = os.listdir(samples_directory) 

output_list_file = "/home/laura/andrea/scripts/TB_DDBB_Building/DDBB_Samples_list"

with open(output_list_file, "w") as f:
    f.write("\n".join(dir_list))

# Leer el listado total de muestras
with open(output_list_file, "r") as f:
    total_samples = set(f.read().splitlines())

total_samples_lenght = len(total_samples)
print("Número total de muestras presentes en la BBDD:")
print(total_samples_lenght)
print("--------------------------------------------------")
# Cálculo del 1% de muestras totales de la BBDD
sampling_samples = max(1, int(0.01 * total_samples_lenght)) 
print("1% del total de muestras:")
print(sampling_samples)
print("--------------------------------------------------")
print("Número de muestras por paquete para la validación:")
print(sampling_samples / 222)

Número total de muestras presentes en la BBDD:
110899
--------------------------------------------------
1% del total de muestras:
1108
--------------------------------------------------
Número de muestras por paquete para la validación:
4.990990990990991


In [ ]:
pack_number = 1

for pack_file in sorted(os.listdir(packs_directory)):

    # Leer el listado total de 500 muestras en cada paquete
    with open(output_list_file, "r") as f:
        total_samples = set(f.read().splitlines())

    # Convertir en lista para evitar conflicto con random.sample
    total_samples_list = list(total_samples)

    # Seleccionar 5 muestras aleatorias por paquete
    validation_samples = set(random.sample(total_samples_list, 5))

    pack_file = os.path.join(validation_directory, f"Validation_Samples_pack_{pack_number}")

    with open(pack_file, "w") as f:
        f.write("\n".join(validation_samples))

    pack_number += 1

In [ ]:
final_output_file = os.path.join(validation_directory, "All_Validation_Samples")

with open(final_output_file, "w") as outfile:

    for file in sorted(os.listdir(validation_directory)):

        if file.startswith("Validation_Samples_pack_"):

            file_path = os.path.join(validation_directory, file)
            
            with open(file_path, "r") as infile:
                outfile.write(infile.read())
                outfile.write("\n")  # Añadir salto de línea entre archivos si no está

In [ ]:
with open(final_output_file, "r") as f:
    total_samples = set(f.read().splitlines())

# Para cada muestra en la lista, busca directamente su carpeta
for sample_id in total_samples:
    sample_folder = os.path.join(samples_directory, sample_id)

    print(sample_folder)
    
    if not os.path.isdir(sample_folder):
        print(f'Carpeta no encontrada para muestra: {sample_id}')
        continue

    tsv_file = os.path.join(sample_folder, "snps.all.ivar.tsv")

    if not os.path.isfile(tsv_file):
        print(f'Archivo TSV no encontrado para muestra: {sample_id}')
        continue

    try:
        df_file_sample = pd.read_csv(tsv_file, sep='\t')

        df_file_sample = df_file_sample.drop(columns=['ID', 'ALT_QUAL', 'FILTER', 'REF_DP', 
                                                'ALT_DP', 'REF_FREQ', 'OLDVAR', 'REGION'], errors = 'ignore')

        # Filtrado de SNPs por calidad:
        df_file_sample = df_file_sample[
            (df_file_sample['TOTAL_DP'] >= 20) & 
            (df_file_sample['ALT_FREQ'] >= 0.7) & 
            (df_file_sample['TYPE'] == "snp")]      

        # Se guardan solo los SNPs comunes entre la muestra y los de la BBDD
        matched_snps = df_file_sample.merge(df_filtered_SNPs, on=['POS', 'REF', 'ALT'], how='inner').drop(columns=['REGION'], errors='ignore')

        matched_snps['CHECK_SAMPLES'] = None

        for index, row in matched_snps.iterrows():
            if sample_id in row['SAMPLES']:
                matched_snps.at[index, 'CHECK_SAMPLES'] = f"{sample_id}-OK"

        # Agregar columna de conteo de chequeo
        matched_snps['CHECK_SAMPLES_COUNT'] = matched_snps['CHECK_SAMPLES'].apply(lambda x: len(x.split(", ")) if pd.notna(x) else 0)

        output_file = os.path.join(output_directory, f'SNPs_Validation_{sample_id}.tsv')
        matched_snps.to_csv(output_file, sep='\t', index=False)

        print(matched_snps)
        
    except Exception as e:
                print(f'Error procesando {sample_id}: {e}')

/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/SRR26788508
          POS REF ALT  TOTAL_DP TYPE  ALT_FREQ  \
0        1849   C   A        49  snp       1.0   
1        2532   C   T        38  snp       1.0   
2        9143   C   T        61  snp       1.0   
3       11820   C   G        41  snp       1.0   
4       13460   G   A        44  snp       1.0   
...       ...  ..  ..       ...  ...       ...   
1069  4395387   G   A        40  snp       1.0   
1070  4397736   C   T        43  snp       1.0   
1071  4401732   C   A        38  snp       1.0   
1072  4407927   T   G        36  snp       1.0   
1073  4408920   G   A        49  snp       1.0   

                                                SAMPLES  COUNT  \
0     DRR019435, DRR019436, DRR019439, DRR034350, DR...  47800   
1     20190732, DRR019439, DRR034350, DRR034374, DRR...  93060   
2     20190732, DRR019439, DRR034350, DRR034371, DRR...  99283   
3     DRR019435, DRR019436, DRR019439, DRR034350

In [ ]:
final_output_file = os.path.join(validation_directory, "All_Validation_Samples")

with open(final_output_file, "r") as f:
    total_samples = set(f.read().splitlines())

final_validation_file = pd.DataFrame()

results = []  # Lista de diccionarios que formará el DataFrame final

# Para cada muestra en la lista, busca directamente su carpeta
for sample_id in total_samples:
    print(f"Procesando muestra: {sample_id}")
    
    # Ruta a archivo tsv original
    sample_folder = os.path.join(samples_directory, sample_id)
    print(sample_folder)
    
    if not os.path.isdir(sample_folder):
        print(f'Carpeta no encontrada para muestra: {sample_id}')
        continue
    
    # Archivo tsv original de cada muestra:
    tsv_file = os.path.join(sample_folder, "snps.all.ivar.tsv")

    if not os.path.isfile(tsv_file):
        print(f'Archivo TSV no encontrado para muestra: {sample_id}')
        continue
    
    # Archivo de validación de cada muestra:
    tsv_file_validation = os.path.join(output_directory, f"SNPs_Validation_{sample_id}.tsv")

    if not os.path.isfile(tsv_file_validation):
        print(f'Archivo de validación TSV no encontrado para muestra: {sample_id}')
        continue

    try:
        # Lectura archivo original
        df_file_sample = pd.read_csv(tsv_file, sep='\t', engine='python')
        
        df_file_sample = df_file_sample.drop(columns=['ID', 'ALT_QUAL', 'FILTER', 'REF_DP', 
                                                'ALT_DP', 'REF_FREQ', 'OLDVAR', 'REGION'], errors = 'ignore')

        # Filtrado de SNPs por calidad:
        df_file_sample = df_file_sample[
            (df_file_sample['TOTAL_DP'] >= 20) & 
            (df_file_sample['ALT_FREQ'] >= 0.7) & 
            (df_file_sample['TYPE'] == "snp")]

        # Lectura archivo validación
        sample_validation_df = pd.read_csv(tsv_file_validation, sep='\t', engine='python')
    
        # Comparar si el archivo tsv original y el de validación tienen el mismo número de SNPs
        same_number_snps = "YES" if len(df_file_sample) == len(sample_validation_df) else "NO"
        print(f"Mismo número de SNPs: {same_number_snps}")

        # Validación de SNPs
        check_status = "OK"

        for index, row in sample_validation_df.iterrows():
            if (row['CHECK_SAMPLES'] != f"{sample_id}-OK" and row['CHECK_SAMPLES_COUNT'] != 1) in df_file_sample:
                check_status = "Fail"
                print("Hay SNPs no coincidentes.")
                print(row['POS'], row['REF'], row['ALT'])
            
        if check_status == "OK":
            print("Todos los SNPs coinciden. Muestra validada.")

        results.append({'SAMPLE_ID': sample_id, 'Same_Number_SNPs': same_number_snps, 'Check_Sample': check_status})

    except Exception as e:
        print(f"Error procesando la muestra {sample_id}: {e}")
        results.append({'SAMPLE_ID': sample_id, 'Same_Number_SNPs': 'ERROR', 'Check_Sample': 'ERROR'})

# Guardar el diccionario en un dataframe
final_validation_file = pd.DataFrame(results)

output_file = os.path.join(validation_directory, f'Final_TB_DDBB_Validation.tsv')
final_validation_file.to_csv(output_file, sep='\t', index=False)

Procesando muestra: SRR12882329
/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/SRR12882329
Mismo número de SNPs: YES
Todos los SNPs coinciden. Muestra validada.
Procesando muestra: ERR10112875
/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/ERR10112875
Mismo número de SNPs: YES
Todos los SNPs coinciden. Muestra validada.
Procesando muestra: SRR28859758
/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/SRR28859758
Mismo número de SNPs: YES
Todos los SNPs coinciden. Muestra validada.
Procesando muestra: ERR6358861
/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/ERR6358861
Mismo número de SNPs: YES
Todos los SNPs coinciden. Muestra validada.
Procesando muestra: ERR5864736
/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants/ERR5864736
Mismo número de SNPs: YES
Todos los SNPs coinciden. Muestra validada.
Procesando muestra: 
/media/NASII/Datos/ANALYSIS/Proyectos/